In [128]:
import tweepy

In [129]:
f = open('secret.txt', 'r')
mess = f.read().split('\n')
consumer_key = mess[0]
consumer_secret = mess[1]
access_token = mess[2]
access_token_secret = mess[3]
f.close()

In [130]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [182]:
import urllib
import datetime
import csv
from bs4 import BeautifulSoup
import time

In [168]:
class Tweet(object):
    def __init__(self, tweet_id, tweet_text):
        self.id = tweet_id
        self.text = tweet_text

In [167]:
def get_tweets(query):
    
    # Craft the query
    params = {
        'f': 'tweets',
        'q': query,
        'lang': 'en',
    }

    q_string = urllib.parse.urlencode(params)
    url = 'https://twitter.com/search?' + q_string
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
    req = urllib.request.Request(url, headers=headers)
    resp = urllib.request.urlopen(req)
    
    # Scrape the tweets
    soup = BeautifulSoup(resp.read(), "html.parser")
    tweets = soup.find_all('li','js-stream-item')
    
    tweet_list = []
    for tweet in tweets:
        if tweet.find('p', 'TweetTextSize'):
            tweet_text = tweet.find('p','TweetTextSize').text.encode('utf8')
            tweet_id = tweet['data-item-id']
            tweet_list.append(Tweet(tweet_id, tweet_text))
        else:
            continue
            
    return tweet_list

In [181]:
bernie_keywords = ['Bernie', 'Sanders', 'BernieSanders', 'Feel the Bern', 'Bern', 'feelthebern']
rubio_keywords = ['Marco', 'Rubio', 'marcorubio', '"A New American Century"']
cruz_keywords = ['Ted', 'Cruz', 'tedcruz', 'CruzCrew', '"Together, we will win"']
clinton_keywords = ['Hilary', 'Clinton', 'HilaryClinton', '"I\'m with her"', '"Stronger Together"', 'imwithher']
trump_keywords = ['Donald', 'Trump', 'DonaldTrump', 'NeverTrump', '"Make America Great Again"', 'maga']

primaries = {'Democrat': [('Clinton', clinton_keywords), 
                          ('Sanders', bernie_keywords)],
             'Republican': [('Rubio', rubio_keywords),
                            ('Cruz', cruz_keywords),
                            ('Trump', trump_keywords)]}

In [162]:
# Generate the query string
def geo_time_query_part(latitude, longitude, radius, after, before):
    since = datetime.datetime.strptime(after, '%m/%d/%y')
    until = datetime.datetime.strptime(before, '%m/%d/%y')
    
    geo = ' geocode:%.6f,%.6f,%dkm' % (float(latitude), float(longitude), float(radius))
    time_part = (' since:%s' % since.strftime('%Y-%m-%d')) + (' until:%s' % until.strftime('%Y-%m-%d'))
    
    return time_part + geo
    

In [163]:
# Generate all the time-based and geo-based components from the csv
state_dict = dict()

with open('data/raw_calc_lat_lon.csv', 'r') as csvfile:
    tab = csv.reader(csvfile, delimiter=',')
    for row in tab:
        if row[1] != 'latitude':
            state = row[0]
            latitude = row[1]
            longitude = row[2]
            radius = row[3]
            date = row[4]
            party = row[5]
            after = row[6]
            until = row[7]
            
            if state not in state_dict:
                state_dict[state] = dict()
            
            state_dict[state][party] = geo_time_query_part(latitude, longitude, radius, after, until)
            

In [183]:
# Get all the tweets for a person
qs_list = []
tweets = dict()
for state in state_dict.keys():
    tweets[state] = dict()
    
    for primary in primaries.keys():
        tweets[state][primary] = dict()
        for candidate in primaries[primary]:
            geo_time = state_dict[state][primary]
            query_string = " OR ".join(candidate[1]) + geo_time
            
            time.sleep(1)
            dl_tweets = get_tweets(query_string)
            tweets[state][primary][candidate[0]] = dl_tweets
            print('%s %s %s: Downloaded %d tweets.' % (state, primary, candidate[0], len(dl_tweets)))
            

Alabama Democrat Clinton: Downloaded 20 tweets.
Alabama Democrat Sanders: Downloaded 0 tweets.
Alabama Republican Rubio: Downloaded 20 tweets.
Alabama Republican Cruz: Downloaded 20 tweets.
Alabama Republican Trump: Downloaded 20 tweets.
Alaska Democrat Clinton: Downloaded 0 tweets.
Alaska Democrat Sanders: Downloaded 0 tweets.
Alaska Republican Rubio: Downloaded 1 tweets.
Alaska Republican Cruz: Downloaded 6 tweets.
Alaska Republican Trump: Downloaded 0 tweets.
Arizona Democrat Clinton: Downloaded 20 tweets.
Arizona Democrat Sanders: Downloaded 2 tweets.
Arizona Republican Rubio: Downloaded 4 tweets.
Arizona Republican Cruz: Downloaded 20 tweets.
Arizona Republican Trump: Downloaded 20 tweets.
Arkansas Democrat Clinton: Downloaded 15 tweets.
Arkansas Democrat Sanders: Downloaded 0 tweets.
Arkansas Republican Rubio: Downloaded 14 tweets.
Arkansas Republican Cruz: Downloaded 8 tweets.
Arkansas Republican Trump: Downloaded 5 tweets.
California Democrat Clinton: Downloaded 20 tweets.
Cali

In [170]:
qs_list

['Hilary OR Clinton OR HilaryClinton OR "I\'m with her" OR "Stronger Together" OR imwithher since:2016-02-25 until:2016-03-02 geocode:32.841667,-86.633333,146km',
 'Bernie OR Sanders OR BernieSanders OR Feel the Bern OR Bern OR feelthebern since:2016-02-25 until:2016-03-02 geocode:32.841667,-86.633333,146km',
 'Marco OR Rubio OR marcorubio OR "A New American Century" since:2016-02-25 until:2016-03-02 geocode:32.841667,-86.633333,146km',
 'Ted OR Cruz OR tedcruz OR CruzCrew OR "Together, we will win" since:2016-02-25 until:2016-03-02 geocode:32.841667,-86.633333,146km',
 'Donald OR Trump OR DonaldTrump OR NeverTrump OR "Make America Great Again" OR maga since:2016-02-25 until:2016-03-02 geocode:32.841667,-86.633333,146km',
 'Hilary OR Clinton OR HilaryClinton OR "I\'m with her" OR "Stronger Together" OR imwithher since:2016-03-21 until:2016-03-27 geocode:64.731667,-152.470000,523km',
 'Bernie OR Sanders OR BernieSanders OR Feel the Bern OR Bern OR feelthebern since:2016-03-21 until:2016

In [190]:
for twt in tweets['Texas']['Republican']['Cruz']:
    print(twt.id)

704807367163052032
704805365523701760
704768318175969280
704767968635277312
704765140629991424
704764180566556673
704755625264181248
704754716043968512
704733669018173440
704730200202878976
704730198223114240
704729395009863682
704729169456996352
704728873095790593
704711274651713536
704709199809421312
704693208807292928
704677016042401794
704660409974923264
704626504043331588


In [172]:
ts = get_tweets(qs_list[0])    

In [180]:
ts[5].text

b'Maybe his new plan is to be Hillary Clinton\xe2\x80\x99s VP choice. https://twitter.com/ZekeJMiller/status/704397766584303616\xc2\xa0\xe2\x80\xa6'